In [1]:
import torchtext
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
get_tokenizer??

In [2]:
data_dir = 'imdb/train'
test_dir = 'imdb/test'
import torchtext.datasets as datasets
import os

In [3]:
def read_data(data_dir, review=None):
    reviews = [review] * 12500#len(os.listdir(data_dir))
    path = [os.path.join(data_dir, p) for p in os.listdir(data_dir)]
    txt = list(map(lambda x:open(x, encoding='utf-8').read(), path))
    return zip(reviews, txt)

In [4]:
pos_rev  = list(read_data(data_dir+'/pos',review=1))
neg_rev = list(read_data(data_dir+'/neg', review=0))

In [5]:
test_pos = list(read_data(test_dir+'/pos',review=1))
test_neg = list(read_data(test_dir+'/neg',review=0))

In [6]:
test_reviews = list(iter(test_pos+test_neg))

In [166]:
len(test_reviews)

25000

In [7]:
reviews = list(iter(pos_rev+neg_rev))

In [8]:
tokenizer = get_tokenizer('basic_english')

In [9]:

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)



In [10]:
vocab = build_vocab_from_iterator(yield_tokens(reviews), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [11]:
len(vocab)

100683

In [12]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

In [21]:
text_pipeline('here is the an example')

[131, 9, 1, 40, 464]

In [23]:
get_tokens = lambda x: vocab.lookup_tokens(x)

In [24]:
get_tokens([131, 9, 1, 40, 464])

['here', 'is', 'the', 'an', 'example']

In [14]:
import torch

In [15]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [16]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)



In [17]:
train_iter = reviews
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [18]:
dl = next(iter(dataloader))

In [61]:
torch.tensor(of[:-1]).cumsum(dim=0)

tensor([ 178,  677,  849,  993, 1127, 1336, 1458])

In [19]:
label,text,offset=dl

In [20]:
text.shape

torch.Size([1874])

In [41]:
get_tokens(tx)[:10]

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', '.', 'it', 'ran', 'at']

In [42]:
offset

tensor([   0,  178,  677,  849,  993, 1127, 1336, 1458])

In [46]:
get_tokens([178])

['lot']

In [ ]:
get_tokens()

In [155]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
1,i
1000,64

In [156]:
#train_iter = reviews
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [159]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [167]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training



In [171]:
len(list(to_map_style_dataset(train_iter)))

25000

In [172]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = reviews, test_reviews
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)



In [174]:
l,t,o=next(iter(train_dataloader))

In [179]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time: 64.34s | valid accuracy    0.772 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time: 63.21s | valid accuracy    0.711 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time: 61.66s | valid accuracy    0.836 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time: 68.93s | valid accuracy    0.839 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time: 64.76s | valid accuracy    0.843 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time: 68.34s |

In [180]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.848


In [201]:
ag_news_label = {0: "negative",
                 1: "positive"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()
        #return output

In [205]:
ex_text_str = "Nice movie"

model = model.to("cpu")

#print("This is a %s review" %ag_news_label[predict(ex_text_str, text_pipeline)])
print("This is a %s review" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a positive review
